In [3]:
import pandas as pd

# ===============================
#    Funções Auxiliares (Helpers)
# ===============================

    """
    Loader flexível para os masters do MERGE Dataset.
    
    Parâmetros:
        path (str): caminho do master_metadata (ex: 'metadata/master_metadata_audio_balanced.csv')
        split (str): 'train', 'val', 'test', ou None (todos)
        split_type (str): 'split_70_15_15' ou 'split_40_30_30'
        quadrant (int or list): 1,2,3,4 ou lista deles, ou None (todos)
        year (int or list): ano ou lista de anos (ex: 2020 ou [2019,2020]), ou None
        artist (str or list): nome do artista ou lista (case-insensitive, match exato ou parcial)
        title (str or list): nome do título ou lista (case-insensitive, match exato ou parcial)
        genres (str or list): gênero(s) a buscar (parcial, case-insensitive)
        return_cols (list): retorna apenas estas colunas (default: todas)
        as_copy (bool): retorna uma cópia do DataFrame
    Retorna:
        pd.DataFrame: dados filtrados
    """

In [4]:
def _filter_by_column(df, value, columns):
    """Filtra o DataFrame por qualquer uma das colunas possíveis, suporta listas e case-insensitive."""
    for col in columns:
        if col in df.columns:
            if isinstance(value, (list, tuple, set)):

                # Para listas: comparação exata (exceto strings)
                if df[col].dtype == object:
                    return df[df[col].str.lower().isin([str(v).lower() for v in value])]
                else:
                    return df[df[col].isin(value)]
            else:
                # Para strings: busca parcial e case-insensitive
                if isinstance(value, str):
                    return df[df[col].str.lower().str.contains(value.lower(), na=False)]
                else:
                    return df[df[col] == value]
    return df

# ===============================
#    Loader Principal
# ===============================

In [5]:
def load_merge_master(
    path,
    split=None,
    split_type='split_70_15_15',
    quadrant=None,
    year=None,
    artist=None,
    title=None,
    genres=None,
    return_cols=None,
    as_copy=True
):
    """
    Loader flexível para os masters do MERGE Dataset.

    Parâmetros detalhados em schema/data_dictionary.md.
    """

    df = pd.read_csv(path)
    # Padroniza as colunas para minúsculas para consistência
    df.columns = [c.lower() for c in df.columns]

    # Filtros principais
    if split is not None and split_type in df.columns:
        df = df[df[split_type] == split]

    if quadrant is not None and 'quadrant' in df.columns:
        if isinstance(quadrant, (list, tuple, set)):
            df = df[df['quadrant'].isin(quadrant)]
        else:
            df = df[df['quadrant'] == quadrant]

    if year is not None:
        df = _filter_by_column(df, year, ['year', 'actualyear'])

    if artist is not None:
        df = _filter_by_column(df, artist, ['artist'])

    if title is not None:
        df = _filter_by_column(df, title, ['title'])

    if genres is not None:
        df = _filter_by_column(df, genres, ['genres'])

    # Seleção de colunas finais (opcional)
    if return_cols:
        # Filtra apenas as colunas que realmente existem no DataFrame
        existing_cols = [col for col in return_cols if col in df.columns]
        df = df[existing_cols]

    return df.copy() if as_copy else df